In [196]:
'''

@Author: Vighnesh Harish Bilgi
@Date: 2022-11-22
@Last Modified by: Vighnesh Harish Bilgi
@Last Modified time: 2022-11-22
@Title : Fetch and Process Tweet items from DynamoDB table using PySpark

'''

'\n\n@Author: Vighnesh Harish Bilgi\n@Date: 2022-11-22\n@Last Modified by: Vighnesh Harish Bilgi\n@Last Modified time: 2022-11-22\n@Title : Fetch and Process Tweet items from DynamoDB table using PySpark\n\n'

In [197]:
import findspark
findspark.init() 

In [198]:
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [199]:
import os
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get('test1_access_key')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.environ.get('test1_secret_access_key')
TABLE_NAME = 'WorldCup2022Tweets'

In [200]:
spark = SparkSession.builder.master("local").appName("Reading DynamoDB items into RDD ").getOrCreate()
print(spark.sparkContext.appName)

Reading DynamoDB items into RDD 


### Reading items from DynamoDB Table into a list of dictionaries

In [201]:
dyDB =  boto3.resource('dynamodb')   

table = dyDB.Table(TABLE_NAME)

table_details = table.scan()
table_items = table_details['Items']

list_of_dict = []
# appending all items/dictionaries from the table to a list
for item in table_items:
    # print(item)
    list_of_dict.append(item)

Prinitng Every Item in the table:


In [202]:
spark_df = spark.createDataFrame(list_of_dict)
spark_df.show(10)

C:\apps\spark-3.0.0-bin-hadoop2.7\python\pyspark\sql\session.py:378: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------------------+--------------------+-------------------+--------------------+-------------------+---------------+--------------------+-----------------------+
|    ArrivalTimestamp|        display_name|           tweet_id|          tweet_text|            user_id|      user_name|  word_count_on_text|word_occurences_on_text|
+--------------------+--------------------+-------------------+--------------------+-------------------+---------------+--------------------+-----------------------+
|2022-11-22 12:51:...|                Rags|1594954296491409409|@INSIGHTUK2 Even ...|1456973849082224646|      rhalharvi|19.00000000000000...|   19.00000000000000...|
|2022-11-22 12:54:...|         Andy Newton|1594954908859596800|World Cup Betting...|           68403698| NewtsDailyLays|16.00000000000000...|   16.00000000000000...|
|2022-11-22 12:50:...|        MODI FOREVER|1594953972326227968|RT @gnuseibeh: Qa...|          163162895| royprateek2010|24.00000000000000...|   24.00000000000000...|
|202

In [203]:
spark_df.describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------------+
|summary|    ArrivalTimestamp|        display_name|            tweet_id|          tweet_text|             user_id|    user_name|  word_count_on_text|word_occurences_on_text|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------------+
|  count|                1004|                1004|                1004|                1004|                1004|         1004|                1004|                   1004|
|   mean|                null|                null|1.594953288752600...|                null|1.082297965066966...|         null|21.49302788844621...|   21.49302788844621...|
| stddev|                null|                null|1.034527046427790...|                null|5.975100107108737...|         null|  

In [204]:
spark_df.summary().show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------------+
|summary|    ArrivalTimestamp|        display_name|            tweet_id|          tweet_text|             user_id|    user_name|  word_count_on_text|word_occurences_on_text|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------------------+
|  count|                1004|                1004|                1004|                1004|                1004|         1004|                1004|                   1004|
|   mean|                null|                null|1.594953288752600...|                null|1.082297965066966...|         null|21.49302788844621...|   21.49302788844621...|
| stddev|                null|                null|1.034527046427790...|                null|5.975100107108737...|         null|  

### Data Processing

In [205]:
# Dropping last column
spark_df = spark_df.drop("word_occurences_on_text")
spark_df.printSchema()

root
 |-- ArrivalTimestamp: string (nullable = true)
 |-- display_name: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- word_count_on_text: decimal(38,18) (nullable = true)



In [206]:
# Changing data types of 'word count on text' to integer
spark_df = spark_df.withColumn("word_count_on_text",spark_df.word_count_on_text.cast('int'))
spark_df.printSchema()

root
 |-- ArrivalTimestamp: string (nullable = true)
 |-- display_name: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- word_count_on_text: integer (nullable = true)



In [207]:
# Removing the milliseconds part of ArrivalTimestamp
spark_df = spark_df.withColumn('Timestamp', substring('ArrivalTimestamp', 1,19))
# spark_df = spark_df.withColumn('Timestamp', split(spark_df['ArrivalTimestamp'], '.').getItem(0)).withColumn('milliseconds', split(spark_df['ArrivalTimestamp'], '.').getItem(1))
# spark_df = spark_df.drop("milliseconds")
spark_df.printSchema()
spark_df.select('ArrivalTimestamp','Timestamp').show(10, truncate = False)

root
 |-- ArrivalTimestamp: string (nullable = true)
 |-- display_name: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- word_count_on_text: integer (nullable = true)
 |-- Timestamp: string (nullable = true)

+--------------------------------+-------------------+
|ArrivalTimestamp                |Timestamp          |
+--------------------------------+-------------------+
|2022-11-22 12:51:58.145000+05:30|2022-11-22 12:51:58|
|2022-11-22 12:54:24.462000+05:30|2022-11-22 12:54:24|
|2022-11-22 12:50:40.345000+05:30|2022-11-22 12:50:40|
|2022-11-22 12:46:47.176000+05:30|2022-11-22 12:46:47|
|2022-11-22 12:41:56.082000+05:30|2022-11-22 12:41:56|
|2022-11-22 12:52:24.573000+05:30|2022-11-22 12:52:24|
|2022-11-22 12:48:40.395000+05:30|2022-11-22 12:48:40|
|2022-11-22 12:50:59.093000+05:30|2022-11-22 12:50:59|
|2022-11-22 12:44:58.143000+05:30|2022-11-2

In [208]:
# Convert Timestamp column of type string into type timestamp 
spark_df = spark_df.withColumn("Timestamp_2",to_timestamp("Timestamp"))
spark_df.printSchema()
spark_df.select('ArrivalTimestamp','Timestamp','Timestamp_2').show(10, truncate = False)

root
 |-- ArrivalTimestamp: string (nullable = true)
 |-- display_name: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- word_count_on_text: integer (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Timestamp_2: timestamp (nullable = true)

+--------------------------------+-------------------+-------------------+
|ArrivalTimestamp                |Timestamp          |Timestamp_2        |
+--------------------------------+-------------------+-------------------+
|2022-11-22 12:51:58.145000+05:30|2022-11-22 12:51:58|2022-11-22 12:51:58|
|2022-11-22 12:54:24.462000+05:30|2022-11-22 12:54:24|2022-11-22 12:54:24|
|2022-11-22 12:50:40.345000+05:30|2022-11-22 12:50:40|2022-11-22 12:50:40|
|2022-11-22 12:46:47.176000+05:30|2022-11-22 12:46:47|2022-11-22 12:46:47|
|2022-11-22 12:41:56.082000+05:30|2022-11-22 12:41:56|2022-11-22 12:41:56|
|2

In [209]:
# Dropping ArrivalTimestamp and Timestamp column, renaming Timestamp_2 to Timestamp , reordering columns and then orders DF by Timestamp and then tweet_id
spark_df = spark_df.drop("ArrivalTimestamp", "Timestamp")
spark_df = spark_df.withColumnRenamed("Timestamp_2","Timestamp")
spark_df = spark_df.select("tweet_id","Timestamp","display_name","user_name","user_id","tweet_text","word_count_on_text")
spark_df = spark_df.sort(col("Timestamp"),col("tweet_id"))
spark_df.printSchema()
spark_df.show()

root
 |-- tweet_id: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)
 |-- display_name: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- word_count_on_text: integer (nullable = true)

+-------------------+-------------------+----------------------------------+---------------+-------------------+--------------------+------------------+
|           tweet_id|          Timestamp|                      display_name|      user_name|            user_id|          tweet_text|word_count_on_text|
+-------------------+-------------------+----------------------------------+---------------+-------------------+--------------------+------------------+
|1594951526619176961|2022-11-22 12:40:57|                           resa_ds|      leyla_677| 965136351455715328|RT @AmnestyIran: ...|                23|
|1594951523317981184|2022-11-22 12:40:58|                              O.G.|  wha

In [210]:
spark_rdd = spark_df.rdd